# FLOW-DC: Flexible Large-scale Orchestrated Workflow for Data Collection

This notebook demonstrates the complete FLOW-DC pipeline for distributed image downloading:

1. **Partition** - Split dataset manifest into partitions grouped by URL host
2. **Estimate** - Calculate expected download size
3. **Download** - Run distributed downloads with TaskVine

## Prerequisites

```bash
conda activate FLOW-DC
```

In [ ]:
import os
import json
import pandas as pd

# Verify we're in the right directory
if not os.path.exists('../bin/download_batch.py'):
    os.chdir('..')
    
print(f"Working directory: {os.getcwd()}")

## Step 1: Check Input Data

Verify that your input parquet file exists and has the expected columns.

In [ ]:
# Configure your input file
INPUT_PARQUET = "files/input/dataset.parquet"  # Change this to your input file
URL_COLUMN = "url"  # Change this to your URL column name
LABEL_COLUMN = "species"  # Change this to your label column (or None)

# Load and inspect
df = pd.read_parquet(INPUT_PARQUET)
print(f"Loaded {len(df):,} rows")
print(f"\nColumns: {list(df.columns)}")
print(f"\nSample URLs:")
df[URL_COLUMN].head()

## Step 2: Estimate Download Size

Use the CalcDatasetSize script to estimate how much storage you'll need.

In [ ]:
%%bash -s "$INPUT_PARQUET" "$URL_COLUMN"
# Sample 1000 URLs for quick estimation
python bin/CalcDatasetSize.py --input "$1" --url_column "$2" --sample 1000

## Step 3: Partition Dataset by Host

Split the dataset into partitions grouped by URL host. This optimizes distributed downloading by:
- Keeping URLs from the same host in the same partition
- Balancing partition sizes using greedy bin-packing
- Enabling per-host rate limiting in PolicyBBR

In [ ]:
# Configure partitioning
NUM_PARTITIONS = 10  # Number of worker partitions
OUTPUT_FOLDER = "files/input/partitions"

# Create output directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
%%bash -s "$INPUT_PARQUET" "$URL_COLUMN" "$NUM_PARTITIONS" "$OUTPUT_FOLDER"
python bin/SplitParquet.py \
    --parquet "$1" \
    --url_col "$2" \
    --groups $3 \
    --output_folder "$4" \
    --method host \
    --add_host_column

In [ ]:
# Verify partitions were created
partitions = sorted([f for f in os.listdir(OUTPUT_FOLDER) if f.endswith('.parquet')])
print(f"Created {len(partitions)} partitions:")
for p in partitions:
    pf = pd.read_parquet(os.path.join(OUTPUT_FOLDER, p))
    hosts = pf['host'].nunique() if 'host' in pf.columns else 'N/A'
    print(f"  {p}: {len(pf):,} rows, {hosts} unique hosts")

## Step 4: Configure TaskVine

Create or update the TaskVine configuration file.

In [ ]:
# TaskVine configuration
config = {
    "port_number": 9123,
    "parquets_directory": OUTPUT_FOLDER,
    "output_directory": "files/output/images",
    
    "url_col": URL_COLUMN,
    "label_col": LABEL_COLUMN,
    
    "concurrent_downloads": 1000,
    "timeout_sec": 30,
    "enable_polite_controller": True,
    
    # PolicyBBR settings
    "initial_rate": 100.0,
    "min_rate": 1.0,
    "max_rate": 10000.0,
    "per_host_conc_init": 16,
    "per_host_conc_cap": 512,
    
    # Task settings
    "max_retries": 3,
    "timeout_minutes": 60,
    "task_cores": 4,
    "task_memory_mb": 8000,
    
    # Output settings
    "create_tar": True,
    "output_format": "imagefolder"
}

# Save configuration
config_path = "files/config/taskvine_flowdc.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
    
print(f"Saved configuration to {config_path}")

## Step 5: Start TaskVine Manager

Run the TaskVine manager to distribute download tasks to workers.

**Note:** You need to have TaskVine workers running and connected to the manager port.

In [ ]:
# Dry run to preview tasks
!python bin/TaskvineFLOWDC.py --config files/config/taskvine_flowdc.json --dry_run

In [ ]:
# Start the actual TaskVine manager (uncomment to run)
# !python bin/TaskvineFLOWDC.py --config files/config/taskvine_flowdc.json

## Alternative: Cloud Upload Mode

For large datasets, use the cloud version that uploads directly to cloud storage.

In [ ]:
# Cloud configuration
cloud_config = {
    "port_number": 9123,
    "parquets_directory": OUTPUT_FOLDER,
    
    "cloud_destination": "AIIRA_Dataset",  # Change to your destination
    "cloud_tool": "gocmd",  # Options: gocmd, aws, gsutil, rclone
    
    "url_col": URL_COLUMN,
    "label_col": LABEL_COLUMN,
    
    "concurrent_downloads": 1000,
    "enable_polite_controller": True,
    
    "max_retries": 3,
    "timeout_minutes": 120,
    "task_cores": 8,
    "task_memory_mb": 16000,
}

# Save cloud configuration
cloud_config_path = "files/config/taskvine_flowdc_cloud.json"
with open(cloud_config_path, 'w') as f:
    json.dump(cloud_config, f, indent=2)
    
print(f"Saved cloud configuration to {cloud_config_path}")

In [ ]:
# Dry run cloud tasks
!python bin/TaskvineFLOWDCCloud.py --config files/config/taskvine_flowdc_cloud.json --dry_run

## Step 6: Verify Results

After downloads complete, verify the output.

In [ ]:
# Check output directory
output_dir = config["output_directory"]
if os.path.exists(output_dir):
    tar_files = [f for f in os.listdir(output_dir) if f.endswith('.tar.gz')]
    print(f"Output tar files in {output_dir}:")
    for f in sorted(tar_files):
        size_mb = os.path.getsize(os.path.join(output_dir, f)) / (1024 * 1024)
        print(f"  {f}: {size_mb:.1f} MB")
else:
    print(f"Output directory {output_dir} does not exist yet.")

## Single-Machine Download (Alternative)

For smaller datasets or testing, you can run downloads directly without TaskVine.

In [ ]:
# Single machine download (for testing or small datasets)
# !python bin/download_batch.py --config files/config/gbif.json